In [27]:
import os

In [28]:
model_name = sayan-gpt35trubo16k
os.environ["AZURE_OPENAI_API_KEY"] = "d92ec509f0724079b5902d66692f75d6"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://raviriturajrohansaurabhsayan.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2024-02-01"
#os.environ["HUGGING_FACE_API"] = 

In [29]:
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

response = client.chat.completions.create(
    model="karan-gpt35trubo16k", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, other Azure AI services also support customer managed keys for added security and control. Examples of such services include Azure Cognitive Services and Azure Machine Learning.


## Langchain Approach

In [88]:
from langchain_openai import AzureChatOpenAI

In [89]:
llm = AzureChatOpenAI(
    deployment_name="karan-gpt35trubo16k",
)
llm.invoke("What is Large Language Model?")

AIMessage(content="A large language model refers to a type of artificial intelligence model that is trained on a vast amount of text data to understand and generate human-like language. These models are typically built using deep learning techniques, such as transformers, and are capable of performing a range of natural language processing tasks, including language translation, text generation, question answering, summarization, and more. These models have millions or even billions of parameters, allowing them to capture intricate patterns and nuances in language. Examples of large language models include OpenAI's GPT-3 (Generative Pre-trained Transformer 3) and Google's BERT (Bidirectional Encoder Representations from Transformers).", response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 13, 'total_tokens': 143}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filte

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [90]:
llm = AzureChatOpenAI(
    deployment_name="karan-gpt35trubo16k",
)
result = llm.invoke("What is Large Language Models?")
parser.invoke(result)

"Large language models refer to artificial intelligence models that are trained on vast amounts of data to understand and generate human-like text. These models are typically based on deep learning techniques, specifically using transformer architectures. They are capable of generating coherent and contextually relevant text in response to prompts or queries.\n\nLarge language models have the ability to understand and generate text across various domains and languages. They can perform tasks like language translation, summarization, sentiment analysis, question answering, and even creative writing. These models are trained on massive datasets, often including books, articles, websites, and other textual sources, to acquire a broad understanding of human language patterns and knowledge.\n\nProminent examples of large language models include OpenAI's GPT (Generative Pre-trained Transformer) models, such as GPT-3, which has 175 billion parameters. These models have demonstrated impressive

### Creating chains using LCEL

In [32]:
import os
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [91]:
# Set up the Azure OpenAI model
model = AzureChatOpenAI(
    deployment_name="karan-gpt35trubo16k",
)


### Few-Shot Examples:

1. **Example 1: Introduction to Generative AI**
   - User: "Can you explain what generative AI is?"
     - Assistant: "Generative AI refers to algorithms that generate new content, such as images, text, or even music, based on patterns in data. It's different from other AI approaches because it focuses on creating rather than just analyzing or predicting."

2. **Example 2: Understanding Neural Networks**
   - User: "What are neural networks and how do they work?"
     - Assistant: "Neural networks are a type of machine learning model inspired by the human brain. They consist of interconnected nodes, or neurons, organized in layers. These networks can learn patterns and make decisions based on input data, making them versatile for tasks like image recognition and natural language processing."

3. **Example 3: Applications of Generative AI**
   - User: "Where is generative AI used?"
     - Assistant: "Generative AI finds applications in various fields. For instance, in healthcare, it helps in generating synthetic data for training medical models. In entertainment, it's used to create realistic characters and scenes in movies and games."

4. **Example 4: Training and Evaluation in Machine Learning**
   - User: "How do you train a machine learning model?"
     - Assistant: "Training a model involves feeding it with labeled data, letting it learn patterns, and adjusting its parameters to minimize errors. Evaluation metrics like accuracy and F1 score are then used to measure how well the model performs on new, unseen data."

5. **Example 5: Ethical Considerations in AI Development**
   - User: "What ethical issues should be considered in AI development?"
     - Assistant: "Ethical concerns in AI include issues like bias in training data, privacy implications of data usage, and the potential for automation to impact jobs. It's important for developers to prioritize fairness, transparency, and accountability in AI systems."


In [92]:
examples = [
    {
        "User": "Can you explain what generative AI is?",
        "Assistant": '''Generative AI refers to algorithms that generate new content, such as images, text,or even music, based on patterns in data. 
        It is different from other AI approaches because it focuses on creating rather than just analyzing or predicting.'''
        },
     {
     "User": "How do you train a machine learning model?",
     "Assistant": '''Training a model involves feeding it with labeled data, letting it learn patterns, 
     and adjusting its parameters to minimize errors. Evaluation metrics like accuracy and F1 score are then used to measure how well the model performs on new, unseen data.'''
     }
]

In [93]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{User}"),
        ("ai", "{Assistant}"),
    ]
)

In [94]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    
)

In [95]:
print(few_shot_prompt.format())

Human: Can you explain what generative AI is?
AI: Generative AI refers to algorithms that generate new content, such as images, text,or even music, based on patterns in data. 
        It is different from other AI approaches because it focuses on creating rather than just analyzing or predicting.
Human: How do you train a machine learning model?
AI: Training a model involves feeding it with labeled data, letting it learn patterns, 
     and adjusting its parameters to minimize errors. Evaluation metrics like accuracy and F1 score are then used to measure how well the model performs on new, unseen data.


In [96]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ("human", "{User}"),
    ]
)

In [97]:
print(final_prompt.format(User="What is LLM?"))

System: You are a helpful AI Assistant
Human: Can you explain what generative AI is?
AI: Generative AI refers to algorithms that generate new content, such as images, text,or even music, based on patterns in data. 
        It is different from other AI approaches because it focuses on creating rather than just analyzing or predicting.
Human: How do you train a machine learning model?
AI: Training a model involves feeding it with labeled data, letting it learn patterns, 
     and adjusting its parameters to minimize errors. Evaluation metrics like accuracy and F1 score are then used to measure how well the model performs on new, unseen data.
Human: What is LLM?


In [99]:
# Implement a simple parser to process the response
parser = StrOutputParser()

# Create a chain that uses the prompt, model, and parser
# chain = LLMChain(
#     llm=model,
#     prompt_template=final_prompt,
#     parser=parser
# )

chain = final_prompt | llm | parser

# Generate the response using the chain
response = chain.invoke({"User": "What is Large Language Model?"})

# Print the parsed response
print(response)


A Large Language Model (LLM) refers to a type of artificial intelligence model that is trained to understand and generate human-like text. LLMs are typically trained on massive datasets to learn the statistical patterns and relationships in language. They are capable of generating coherent and contextually relevant text, making them useful for tasks like text completion, translation, summarization, and even generating creative content. Examples of popular LLMs include OpenAI's GPT-3 and Google's BERT.
